# RNN Multiplication

## Imports

In [1]:
from __future__ import print_function

import collections

import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf

from tensorflow.keras import layers

import numpy as np
import tensorflow as tf 
import random
import time

from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec, KeyedVectors, Phrases

import arithmetic_datasets as ad

## Data


In [2]:
num_examples = 20000
rstart = 1
rend = 10000

In [3]:
x_train, y_train = ad.gen_data_mult(num_examples, rstart, rend)
x_test, y_test = ad.gen_data_mult(50, rstart, rend)
rdx = np.array(x_train).flatten()
rdy = np.array(y_train).flatten()

reverse_dictionary = np.unique(np.append(rdx, rdy))

dictionary = {}
for x in range(len(reverse_dictionary)):
    dictionary[reverse_dictionary[x]] = x

In [4]:
x_train = x_train.reshape(num_examples, 3, 1)
x_test = x_test.reshape(50, 3, 1)

print(x_train.shape)
print(x_test.shape)

(20000, 3, 1)
(50, 3, 1)


In [5]:
x_train = np.array([[dictionary[num[0]] for num in example] for example in x_train])
#x_test = np.array([[dictionary[num[0]] for num in example] for example in x_test])
print(np.amax(y_train))

y_train = np.array([dictionary[example] for example in y_train])
#y_test = np.array([[dictionary[num[0]] for num in example] for example in y_test])

x_train = x_train.reshape(num_examples, 3, 1)
x_test = x_test.reshape(50, 3, 1)

print(x_train.shape)
print(x_test.shape)

98573852
(20000, 3, 1)
(50, 3, 1)


## RNN Model

In [9]:
n_hidden = 512
epochs = 50
batch_size = 1

In [14]:
model = tf.keras.Sequential()
# Add an Embedding layer expecting input vocab of size 1000, and
# output embedding dimension of size 64.
model.add(layers.Embedding(len(dictionary), 64, input_length=3))
# model.add(layers.Dense(64, input_shape=(3,10)))
# Add a LSTM layer with 128 internal units.
# model.add(layers.LSTM(128))
new_shape = (3, 1)
# model.add(layers.Dense(64, input_shape=new_shape))


rnn_cell = tf.keras.layers.StackedRNNCells([tf.keras.layers.LSTMCell(n_hidden),tf.keras.layers.LSTMCell(n_hidden)])
# model.add(layers.RNN(rnn_cell, input_length=n_input))
model.add(layers.RNN(rnn_cell, input_shape=new_shape))
# Add a Dense layer with 10 units.
model.add(layers.Dense(len(dictionary), activation="softmax"))

model.summary()

model.compile(optimizer=tf.keras.optimizers.RMSprop(),  # Optimizer
              # Loss function to minimize
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              # List of metrics to monitor
              metrics=['sparse_categorical_accuracy'])

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 3, 64)             1905344   
_________________________________________________________________
rnn_2 (RNN)                  (None, 512)               3280896   
_________________________________________________________________
dense_2 (Dense)              (None, 29771)             15272523  
Total params: 20,458,763
Trainable params: 20,458,763
Non-trainable params: 0
_________________________________________________________________


In [15]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs= epochs)

Epoch 1/50
 3802/20000 [====>.........................] - ETA: 1:02:33 - loss: 10.3011 - sparse_categorical_accuracy: 0.0000e+00

KeyboardInterrupt: 